In [3]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
# from tensorboardX import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler

In [4]:
class SceneDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mode = mode
        self.image_frame = pd.read_csv(csv_file)
        self.img_name = self.image_frame.image_name
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, idx):
        image = cv2.imread(self.root_dir + '/' + self.img_name[idx])
        if self.mode == 'train':
            label = self.image_frame.iloc[idx].label

        if self.transform:
            image = self.transform(Image.fromarray(image))

        if self.mode == 'train':
            return image, label
        else:
            return image

In [12]:
#Input the Training and Testing Data
import cv2
train_path = 'train-scene/train'

train_dataset = SceneDataset(csv_file='train-scene/train.csv',
                                    root_dir=train_path, transform=transforms.Compose([
                                               transforms.RandomHorizontalFlip(),
#                                                transforms.RandomVerticalFlip(),
#                                                transforms.RandomRotation(90),
                                               transforms.Resize((224, 224)),
                                               transforms.RandomRotation(30),
                                               transforms.ToTensor()
                                           ]), mode = 'train')
train_dataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

In [13]:

test_dataset = SceneDataset(csv_file='test.csv', root_dir=train_path, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]), mode = 'val')
test_dataloader = DataLoader(test_dataset, batch_size=32,
                        shuffle=False, num_workers=0)

validation_split = .15
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, 
                                           sampler=train_sampler,shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                                sampler=valid_sampler, shuffle=False, num_workers=0)

In [14]:
dataset_sizes = {'train': len(train_dataset) - split, 'val': split}
print (dataset_sizes)
print (len(train_dataloader), len(test_dataloader), len(train_loader), len(val_loader))

{'train': 14479, 'val': 2555}
533 229 453 80


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import pandas as pd



n_class = 6
kernel_size = (1,1)
stride = (1,1)

""" training functions
"""
model_conv = torchvision.models.densenet121(pretrained='imagenet')
num_ftrs = model_conv.classifier.in_features
maxm = 0
for i, param in model_conv.named_parameters():
    maxm = maxm + 1
    if maxm < 182:
      param.requires_grad = False
print(maxm)
model_conv.classifier = nn.Linear(num_ftrs, 6)
model_conv = model_conv.cuda()

364


In [17]:

criterion = nn.CrossEntropyLoss().cuda()
optimizer_conv = optim.Adam(list(filter(lambda p: p.requires_grad, model_conv.parameters())), lr=5e-4)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_conv,mode = 'min', patience=3, factor = 0.5)




In [24]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, use_gpu, num_epochs=25, mixup = False, alpha = 0.1):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # get the inputs
            for data in tqdm(dataloaders[phase]):
                inputs, labels = data
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                if type(outputs) == tuple:
                    outputs, _ = outputs
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = float(running_loss.cpu()) / dataset_sizes[phase]
            epoch_acc = float(running_corrects.cpu()) / dataset_sizes[phase]


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'phase', epoch_loss, epoch_acc))
#             print (int(running_corrects.cpu()))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
        scheduler.step(epoch_loss)
        for param_group in optimizer.param_groups:
          print(param_group['lr'])


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



In [25]:
model_conv = train_model(model_conv, {'train':train_loader, 'val':val_loader}, dataset_sizes, criterion, optimizer_conv, exp_lr_scheduler, True,
                     num_epochs=30)

Epoch 0/29
----------





  0%|                                                                                          | 0/453 [00:00<?, ?it/s]


  0%|▏                                                                                 | 1/453 [00:00<02:25,  3.10it/s]


  0%|▎                                                                                 | 2/453 [00:00<02:20,  3.20it/s]


  1%|▌                                                                                 | 3/453 [00:00<02:15,  3.32it/s]


  1%|▋                                                                                 | 4/453 [00:01<02:11,  3.42it/s]


  1%|▉                                                                                 | 5/453 [00:01<02:09,  3.46it/s]


  1%|█                                                                                 | 6/453 [00:01<02:06,  3.52it/s]


  2%|█▎                                                                                | 7/453 [00:01<02:04,  3.57it/s]


  2%|█▍              

 15%|███████████▉                                                                     | 67/453 [00:19<01:50,  3.50it/s]


 15%|████████████▏                                                                    | 68/453 [00:19<01:48,  3.53it/s]


 15%|████████████▎                                                                    | 69/453 [00:19<01:48,  3.53it/s]


 15%|████████████▌                                                                    | 70/453 [00:19<01:47,  3.58it/s]


 16%|████████████▋                                                                    | 71/453 [00:20<01:49,  3.48it/s]


 16%|████████████▊                                                                    | 72/453 [00:20<01:48,  3.53it/s]


 16%|█████████████                                                                    | 73/453 [00:20<01:49,  3.48it/s]


 16%|█████████████▏                                                                   | 74/453 [00:21<01:49,  3.45it/s]


 17%|█████████████▍     

 30%|███████████████████████▋                                                        | 134/453 [00:38<01:39,  3.22it/s]


 30%|███████████████████████▊                                                        | 135/453 [00:38<01:38,  3.24it/s]


 30%|████████████████████████                                                        | 136/453 [00:38<01:34,  3.35it/s]


 30%|████████████████████████▏                                                       | 137/453 [00:39<01:34,  3.33it/s]


 30%|████████████████████████▎                                                       | 138/453 [00:39<01:32,  3.41it/s]


 31%|████████████████████████▌                                                       | 139/453 [00:39<01:30,  3.48it/s]


 31%|████████████████████████▋                                                       | 140/453 [00:39<01:28,  3.54it/s]


 31%|████████████████████████▉                                                       | 141/453 [00:40<01:27,  3.56it/s]


 31%|███████████████████

 44%|███████████████████████████████████▍                                            | 201/453 [00:57<01:15,  3.33it/s]


 45%|███████████████████████████████████▋                                            | 202/453 [00:58<01:14,  3.35it/s]


 45%|███████████████████████████████████▊                                            | 203/453 [00:58<01:14,  3.35it/s]


 45%|████████████████████████████████████                                            | 204/453 [00:58<01:13,  3.38it/s]


 45%|████████████████████████████████████▏                                           | 205/453 [00:58<01:12,  3.44it/s]


 45%|████████████████████████████████████▍                                           | 206/453 [00:59<01:12,  3.40it/s]


 46%|████████████████████████████████████▌                                           | 207/453 [00:59<01:13,  3.35it/s]


 46%|████████████████████████████████████▋                                           | 208/453 [00:59<01:12,  3.40it/s]


 46%|███████████████████

...

 89%|██████████████████████████████████████████████████████████████████████▉         | 402/453 [01:53<00:14,  3.61it/s]


 89%|███████████████████████████████████████████████████████████████████████▏        | 403/453 [01:53<00:13,  3.66it/s]


 89%|███████████████████████████████████████████████████████████████████████▎        | 404/453 [01:53<00:13,  3.58it/s]


 89%|███████████████████████████████████████████████████████████████████████▌        | 405/453 [01:53<00:13,  3.64it/s]


 90%|███████████████████████████████████████████████████████████████████████▋        | 406/453 [01:54<00:12,  3.66it/s]


 90%|███████████████████████████████████████████████████████████████████████▉        | 407/453 [01:54<00:12,  3.65it/s]


 90%|████████████████████████████████████████████████████████████████████████        | 408/453 [01:54<00:12,  3.61it/s]


 90%|████████████████████████████████████████████████████████████████████████▏       | 409/453 [01:55<00:12,  3.62it/s]


 91%|███████████████████

phase Loss: 0.0001 Acc: 0.9985





  0%|                                                                                           | 0/80 [00:00<?, ?it/s]


  1%|█                                                                                  | 1/80 [00:00<00:11,  6.87it/s]


  2%|██                                                                                 | 2/80 [00:00<00:11,  6.77it/s]


  4%|███                                                                                | 3/80 [00:00<00:11,  6.42it/s]


  5%|████▏                                                                              | 4/80 [00:00<00:11,  6.47it/s]


  6%|█████▏                                                                             | 5/80 [00:00<00:11,  6.47it/s]


  8%|██████▏                                                                            | 6/80 [00:00<00:11,  6.34it/s]


  9%|███████▎                                                                           | 7/80 [00:01<00:11,  6.55it/s]


 10%|████████▎       

 84%|████████████████████████████████████████████████████████████████████▋             | 67/80 [00:10<00:01,  6.57it/s]


 85%|█████████████████████████████████████████████████████████████████████▋            | 68/80 [00:10<00:01,  6.67it/s]


 86%|██████████████████████████████████████████████████████████████████████▋           | 69/80 [00:10<00:01,  6.66it/s]


 88%|███████████████████████████████████████████████████████████████████████▊          | 70/80 [00:11<00:01,  6.73it/s]


 89%|████████████████████████████████████████████████████████████████████████▊         | 71/80 [00:11<00:01,  6.80it/s]


 90%|█████████████████████████████████████████████████████████████████████████▊        | 72/80 [00:11<00:01,  6.83it/s]


 91%|██████████████████████████████████████████████████████████████████████████▊       | 73/80 [00:11<00:01,  6.89it/s]


 92%|███████████████████████████████████████████████████████████████████████████▊      | 74/80 [00:11<00:00,  6.87it/s]


 94%|███████████████████

phase Loss: 0.0088 Acc: 0.9429
7.8125e-06
Training complete in 70m 7s
Best val Acc: 0.953425


In [26]:
def test_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs, labels_ = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        labels.append(labels_)

    return predictions, labels
  
predictions, labels = test_model(model_conv, val_loader, optimizer_conv, True)




  0%|                                                                                           | 0/80 [00:00<?, ?it/s]


  1%|█                                                                                  | 1/80 [00:00<00:12,  6.15it/s]


  2%|██                                                                                 | 2/80 [00:00<00:12,  6.37it/s]


  4%|███                                                                                | 3/80 [00:00<00:12,  6.32it/s]


  5%|████▏                                                                              | 4/80 [00:00<00:11,  6.45it/s]


  6%|█████▏                                                                             | 5/80 [00:00<00:11,  6.57it/s]


  8%|██████▏                                                                            | 6/80 [00:00<00:10,  6.74it/s]


  9%|███████▎                                                                           | 7/80 [00:01<00:10,  6.67it/s]


 10%|████████▎       

 84%|████████████████████████████████████████████████████████████████████▋             | 67/80 [00:10<00:01,  6.69it/s]


 85%|█████████████████████████████████████████████████████████████████████▋            | 68/80 [00:10<00:01,  6.53it/s]


 86%|██████████████████████████████████████████████████████████████████████▋           | 69/80 [00:10<00:01,  6.54it/s]


 88%|███████████████████████████████████████████████████████████████████████▊          | 70/80 [00:10<00:01,  6.42it/s]


 89%|████████████████████████████████████████████████████████████████████████▊         | 71/80 [00:10<00:01,  6.57it/s]


 90%|█████████████████████████████████████████████████████████████████████████▊        | 72/80 [00:10<00:01,  6.74it/s]


 91%|██████████████████████████████████████████████████████████████████████████▊       | 73/80 [00:11<00:01,  6.78it/s]


 92%|███████████████████████████████████████████████████████████████████████████▊      | 74/80 [00:11<00:00,  6.82it/s]


 94%|███████████████████

In [38]:
model_path = os.getcwd()
torch.save(model_conv.state_dict(), model_path)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\rohit\\Desktop\\ML\\Scene_Classification'

In [39]:
predictions1, labels1 = torch.cat(predictions), torch.cat(labels)
predictions1, labels1 = np.array(predictions1.cpu()), np.array(labels1)

In [40]:
predictions1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [41]:
labels1

array([3, 4, 0, ..., 5, 5, 4], dtype=int64)

In [42]:
len(labels1)

2555

In [48]:
def test1_model(model, dataloaders, optimizer, use_gpu):
    model.train(False)
    predictions = []
    labels = []
    for data in tqdm(dataloaders):
        inputs = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
        else:
            inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        predictions.append(preds)
        #labels.append(labels_)

    return predictions

In [49]:
predictions2 = test1_model(model_conv, test_dataloader, optimizer_conv, True)




  0%|                                                                                          | 0/229 [00:00<?, ?it/s]


  0%|▎                                                                                 | 1/229 [00:00<00:34,  6.55it/s]


  1%|▋                                                                                 | 2/229 [00:00<00:35,  6.40it/s]


  1%|█                                                                                 | 3/229 [00:00<00:35,  6.42it/s]


  2%|█▍                                                                                | 4/229 [00:00<00:34,  6.51it/s]


  2%|█▊                                                                                | 5/229 [00:00<00:36,  6.15it/s]


  3%|██▏                                                                               | 6/229 [00:01<00:38,  5.73it/s]


  3%|██▌                                                                               | 7/229 [00:01<00:40,  5.52it/s]


  3%|██▊             

 29%|███████████████████████▋                                                         | 67/229 [00:10<00:24,  6.73it/s]


 30%|████████████████████████                                                         | 68/229 [00:10<00:24,  6.57it/s]


 30%|████████████████████████▍                                                        | 69/229 [00:10<00:24,  6.58it/s]


 31%|████████████████████████▊                                                        | 70/229 [00:11<00:24,  6.61it/s]


 31%|█████████████████████████                                                        | 71/229 [00:11<00:23,  6.63it/s]


 31%|█████████████████████████▍                                                       | 72/229 [00:11<00:23,  6.65it/s]


 32%|█████████████████████████▊                                                       | 73/229 [00:11<00:23,  6.67it/s]


 32%|██████████████████████████▏                                                      | 74/229 [00:11<00:23,  6.69it/s]


 33%|███████████████████

 59%|██████████████████████████████████████████████▊                                 | 134/229 [00:20<00:14,  6.53it/s]


 59%|███████████████████████████████████████████████▏                                | 135/229 [00:20<00:14,  6.53it/s]


 59%|███████████████████████████████████████████████▌                                | 136/229 [00:21<00:14,  6.53it/s]


 60%|███████████████████████████████████████████████▊                                | 137/229 [00:21<00:14,  6.51it/s]


 60%|████████████████████████████████████████████████▏                               | 138/229 [00:21<00:13,  6.54it/s]


 61%|████████████████████████████████████████████████▌                               | 139/229 [00:21<00:13,  6.54it/s]


 61%|████████████████████████████████████████████████▉                               | 140/229 [00:21<00:13,  6.52it/s]


 62%|█████████████████████████████████████████████████▎                              | 141/229 [00:21<00:13,  6.52it/s]


 62%|███████████████████

 88%|██████████████████████████████████████████████████████████████████████▏         | 201/229 [00:31<00:04,  6.48it/s]


 88%|██████████████████████████████████████████████████████████████████████▌         | 202/229 [00:31<00:04,  6.49it/s]


 89%|██████████████████████████████████████████████████████████████████████▉         | 203/229 [00:31<00:04,  6.47it/s]


 89%|███████████████████████████████████████████████████████████████████████▎        | 204/229 [00:31<00:03,  6.36it/s]


 90%|███████████████████████████████████████████████████████████████████████▌        | 205/229 [00:31<00:03,  6.30it/s]


 90%|███████████████████████████████████████████████████████████████████████▉        | 206/229 [00:31<00:03,  5.76it/s]


 90%|████████████████████████████████████████████████████████████████████████▎       | 207/229 [00:32<00:03,  5.56it/s]


 91%|████████████████████████████████████████████████████████████████████████▋       | 208/229 [00:32<00:03,  5.43it/s]


 91%|███████████████████

In [52]:

predictions3 = torch.cat(predictions2)
predictions3 = np.array(predictions3.cpu())

In [54]:
len(predictions3)

7301

In [59]:
predictions3[:5]

array([5, 0, 4, 2, 5], dtype=int64)

In [57]:
result = pd.read_csv('resnet50_15_col.csv')

In [61]:
result.shape[0]

7301

In [62]:
result['label'] = predictions3

In [63]:
result.head()

,image_name,label
0,3.jpg,5
1,5.jpg,0
2,6.jpg,4
3,11.jpg,2
4,14.jpg,5


In [64]:
result.to_csv('torch_densenet121.csv')